In [1]:
import sympy as sp

In [2]:
def ctensor_tti(A,N,C,L,F,s,i,j,k,l):
    from sympy import KroneckerDelta as Delta 
    s = (A - 2 * N) * Delta(i,j) * Delta(k,l) + N * (Delta(i,k) * Delta(j,l) + Delta(i,l) * Delta(j,k))  + \
        (F-A+2*N) * (Delta(i,j) * s[k] * s[l] + Delta(k,l) * s[i] * s[j])  + \
        (L-N) * (Delta(i,k) * s[j] * s[l] + Delta(i,l) * s[j] * s[k] + Delta(j,k) * s[i] * s[l] + Delta(j,l) * s[i] * s[k])  + \
        (A + C -2 * F - 4 * L) * s[i] * s[j] * s[k] * s[l]
    
    return s

def voigt_index(i, j):
    """
    Maps tensor indices (i, j) to Voigt notation index.
    
    Parameters:
        i (int): Row index (0-based).
        j (int): Column index (0-based).
        
    Returns:
        int: Voigt notation index (0-based).
    """
    if i > j:  # Ensure the indices are in upper triangular order (i <= j)
        i, j = j, i

    mapping = {
        (0, 0): 0,  # sigma_11
        (1, 1): 1,  # sigma_22
        (2, 2): 2,  # sigma_33
        (1, 2): 3,  # sigma_23
        (0, 2): 4,  # sigma_13
        (0, 1): 5   # sigma_12
    }

    return mapping.get((i, j), None)


def ctensor_c21(c21,i,j,k,l):
    m = voigt_index(i,j)
    n = voigt_index(k,l)
    if m > n:
        m,n = n,m
    idx = m * 6 + n - m * (m + 1) // 2

    return c21[idx]


In [3]:
rho,om = sp.symbols('rho,omega',real=True)
z = sp.symbols("z",real=True)
k0 = sp.symbols('k',real=True)

# coordinate system
e3 = sp.Array([0,0,1])
e1 = sp.Array([1,0,0])
e2 = sp.Array([0,1,0])

# axis location
phi = sp.symbols("phi",real=True)
k_hat = sp.Array([sp.cos(phi),sp.sin(phi),0])
t_hat = sp.Array([-sp.sin(phi),sp.cos(phi),0])
k = k_hat * k0

U,V,W = sp.symbols("U,V,W",real=False)
dU,dV,dW = sp.symbols("Udot,Vdot,Wdot",real=False)
I = sp.I
s = V * e3 + e1  * U  + e2  * W 
ds = dV * e3 + e1  * dU + e2  * dW 
s = s.as_mutable()
ds = ds.as_mutable()

# trial function
psi = sp.symbols("psi",real=True)
dpsi = sp.symbols("psidot",real=True)

# c21 vec
c21str = ''
for i in range(6):
    for j in range(i,6):
        c21str += f"c{i+1}{j+1}" + " "
c21vec = sp.symbols(c21str)

# cijkl
c = sp.Array.zeros(3,3,3,3)
c = c.as_mutable()
for i in range(3):
    for j in range(3):
        for p in range(3):
            for q in range(3):
                c[i,j,p,q] = ctensor_c21(c21vec,i,j,p,q)
T = sp.Array([0,0,0])
T = T.as_mutable()
G = T * 1
Lag = 0
for i in range(3):
    a = 0
    b = 0
    Lag = Lag + rho * om**2 * s[i] * sp.conjugate(s[i]) 
    for j in range(3):
        for p in range(3):
            for q in range(3):
                a = a + e3[j] * c[i,j,p,q] * (-I * k[q] * s[p] + e3[q] * ds[p])
                b = b + I * k[j] * c[i,j,p,q] * (-I * k[q] * s[p] + e3[q] * ds[p])

                Lag = Lag - (I * k[j] * sp.conjugate(s[i]) + e3[j] * sp.conjugate(ds[i])) * c[i,j,p,q] *  \
                      (-I * k[q] * s[p] + e3[q] * ds[p])
    T[i] = a
    G[i] = b

out = T * dpsi  + G * psi

In [6]:
sp.diff(Lag,k0)

-2*U*c11*k*cos(phi)**2*conjugate(U) - 2*U*c12*k*sin(phi)*cos(phi)*conjugate(W) + I*U*c13*cos(phi)*conjugate(Vdot) - 2*U*c14*k*sin(phi)*cos(phi)*conjugate(V) + I*U*c14*cos(phi)*conjugate(Wdot) - 2*U*c15*k*cos(phi)**2*conjugate(V) + I*U*c15*cos(phi)*conjugate(Udot) - 4*U*c16*k*sin(phi)*cos(phi)*conjugate(U) - 2*U*c16*k*cos(phi)**2*conjugate(W) - 2*U*c26*k*sin(phi)**2*conjugate(W) + I*U*c36*sin(phi)*conjugate(Vdot) - 2*U*c46*k*sin(phi)**2*conjugate(V) + I*U*c46*sin(phi)*conjugate(Wdot) - 2*U*c56*k*sin(phi)*cos(phi)*conjugate(V) + I*U*c56*sin(phi)*conjugate(Udot) - 2*U*c66*k*sin(phi)**2*conjugate(U) - 2*U*c66*k*sin(phi)*cos(phi)*conjugate(W) - I*Udot*c15*cos(phi)*conjugate(U) - I*Udot*c25*sin(phi)*conjugate(W) - I*Udot*c45*sin(phi)*conjugate(V) - I*Udot*c55*cos(phi)*conjugate(V) - I*Udot*c56*sin(phi)*conjugate(U) - I*Udot*c56*cos(phi)*conjugate(W) - 2*V*c14*k*sin(phi)*cos(phi)*conjugate(U) - 2*V*c15*k*cos(phi)**2*conjugate(U) - 2*V*c24*k*sin(phi)**2*conjugate(W) - 2*V*c25*k*sin(phi)*cos(ph

In [66]:
def replace_fortran_str(text):
    #temp:str = text.replace("+ &\n      "," + ")
    temp:str = text.replace(" &\n      ","")
    temp = temp.replace("+"," + ")
    temp = temp.replace("-"," - ")
    temp = temp.replace("  -"," -")
    temp = temp.replace("  +"," +")
    temp = temp.replace("-  ","-  ")
    temp = temp.replace("+  ","+ ")
    #temp:str = temp.replace("- &\n      "," - ")
    

    # convert to list
    a = list(temp)

    # find all operators
    idx = [i for i in range(len(temp)) if temp.startswith(' + ', i)]
    idx_m = [i for i in range(len(temp)) if temp.startswith(' - ', i)]
    idx.extend(idx_m)
    idx.sort()
    idx.append(len(a) - 1)
    idx1 = [False for i in range(len(idx))]
    
    # add change line symbol at +/-
    for i in range(80,len(a),72):
        # loc = 
        # loc = -1
        # for j in range(len(idx) - 1):
        #     if i >= idx[j] and i < idx[j+1]:
        #         loc = j 
        #         break
        # if loc != -1 and idx1[loc] == False :
        #     s = a[idx[loc]]
        #     if s == ' ':
        #         a[idx[loc]] = ' &\n      '
        #     else:
        #         a[idx[loc]] = s + ' &\n      '
        #     idx1[loc] = True
        # find closest
        loc = -1; dist = 999999
        for j in range(len(idx) - 1):
            id = abs(i - idx[j])
            if dist > id:
                loc = j
                dist = id
        if loc != -1 and idx1[loc] == False :
            s = a[idx[loc]]
            if s == ' ':
                a[idx[loc]] = ' &\n      '
            else:
                a[idx[loc]] = s + ' &\n    '
            idx1[loc] = True
        
    # replace the last &\n with \n
    if ' &\n      ' in a[-1]:
        a[-1] = a[-1].replace(' &\n      ','')
    
    # now replace sin/cos function with corresponding arrays
    f_temp =  ''.join(a)
    f_temp = f_temp.replace("cos(theta0)","costh0")
    f_temp = f_temp.replace("sin(theta0)","sinth0")
    f_temp = f_temp.replace("cos(dphi)","cosphi")
    f_temp = f_temp.replace("sin(dphi)","sinphi")
    f_temp = f_temp.replace("+  ","+ ")
    f_temp = f_temp.replace("-  ","- ")
    f_temp = f_temp.replace("cmplx(0,1)","imag_i")


    return f_temp

In [67]:
funclist = {
    "re":'real',
    "im": "aimag",
}

c21f = ''
kc21f = ''
for i in range(6):
    for j in range(i,6):
        if i == 5 and j == 5:
            c21f += f"c{i+1}{j+1}"
            kc21f += f"Kc{i+1}{j+1}"
        else:
            c21f += f"c{i+1}{j+1}" + ","
            kc21f += f"Kc{i+1}{j+1}" + ","
        

# open a txt file to write all functions
fileid = open("./aniso_subs.f90","w")
fileid.write("!===========================================================================\n")
fileid.write("!============================= AUTO CODE FROM SYMPY ========================\n")
fileid.write("!===========================================================================\n")
fileid.write("\n")

# loop each comp of out
for i in range(3):

    fileid.write(f"subroutine get_comp{i + 1} (NGL,{c21f},jaco, &\n")
    fileid.write(f"                        weight,hp,hpT,K0U,K0V,K0W,K1U,K1V,K1W,&\n")
    fileid.write(f"                        K2U,K2V,K2W) bind(c,name='get_comp{i+1}_')\n")
    fileid.write("  use iso_c_binding,only: c_int,dp => c_double ,dcp => c_double_complex\n")
    fileid.write(f"  implicit none\n\n")
    fileid.write(f"  integer(c_int),value,intent(in)         :: NGL\n")
    fileid.write(f"  real(dp),dimension(NGL),intent(in)  :: {c21f}\n")
    fileid.write(f"  real(dp),dimension(NGL),intent(in)  :: weight\n")
    fileid.write(f"  real(dp),dimension(NGL,NGL),intent(in)  :: hp,hpT\n")
    fileid.write(f"  real(dp),value :: jaco \n")
    fileid.write(f"  complex(dcp),dimension(NGL,NGL),intent(out) :: K1U,K1V,K1W,K2U,K2V,K2W,K0U,K0V,K0W \n")
    fileid.write(f"  \n  !local vars\n")
    fileid.write(f"  integer(c_int)        :: i,j \n")
    fileid.write(f"  complex(dcp),dimension(NGL)  :: temp\n")
    fileid.write(f"  complex(dcp),parameter :: imag_i = cmplx(0.0,1.0,kind=dcp)\n")
    fileid.write(f"\n  !init matrices\n")
    for p in range(3):
        fileid.write(f"  K{p}U(:,:) = (0.0,0.0); K{p}V(:,:) = (0.0,0.0); K{p}W(:,:) = (0.0,0.0);\n")
    fileid.write(f"\n")

    # as polynomial of k and loop k^0,k^1,k^2
    expr = out[i].as_poly(k0)
    for p in range(3): # 
        k_power_expr = expr.coeff_monomial(k0**p)
        
        # as polynomial of field U,V,W,dV,dU,dW
        field_expr_poly = k_power_expr.as_poly(U,V,W,dV,dU,dW)
        f = [[U,dU],
             [W,dW],
             [V,dV]]
        psi_lst = [psi,dpsi]
        for ii in range(3):
            for jj in range(2):
                # weak form
                weak_poly = field_expr_poly.coeff_monomial(f[ii][jj]).as_poly(psi,dpsi)
                for kk in range(2):
                    weak_expr = weak_poly.coeff_monomial(psi_lst[kk])
                    if weak_expr == 0:
                        continue
                    print(f"k^{p}, {f[ii][jj]} {psi_lst[kk]} = {weak_expr}")
                    #weak_expr.replace("cos(theta0)","costh0")
                    f_out = sp.fcode(weak_expr,standard=2008,source_format='free',user_functions=funclist,assign_to="temp(:)")
                    #display(f_out)
                    f_out = replace_fortran_str(f_out)
                    
                    # cache temporary arrays
                    fileid.write(f"  ! k^{p}, {f[ii][jj]} {psi_lst[kk]}\n")
                    fileid.write(f"  {f_out}\n")

                    # compute weak form
                    cur_arr = f'K{p}{f[ii][0]}'
                    if jj == 0 and kk == 0: # U psi
                        fileid.write(f'  do i=1,NGL; {cur_arr}(i,i) = {cur_arr}(i,i) + temp(i) * weight(i) * jaco; enddo\n')
                    elif jj == 0 and kk == 1: # U dpsi
                        fileid.write("  do j=1,NGL; do i=1,NGL; \n")
                        fileid.write(f"    {cur_arr}(i,j) = {cur_arr}(i,j) + temp(j) * weight(j) * hpT(i,j)\n")
                        fileid.write("  enddo; enddo; \n")
                        pass
                    elif jj == 1 and kk == 0: # dU psi
                        fileid.write("  do j=1,NGL; do i=1,NGL; \n")
                        fileid.write(f"    {cur_arr}(i,j) = {cur_arr}(i,j) + temp(i) * weight(i) * hp(i,j)\n")
                        fileid.write("  enddo; enddo; \n")
                        pass 
                    else:
                        fileid.write("  do j=1,NGL; do i=1,NGL; \n")
                        fileid.write(f"    {cur_arr}(i,j) = {cur_arr}(i,j) + sum(weight * temp / jaco * hp(:,j) * hp(:,i))\n")
                        fileid.write("  enddo; enddo; \n")
                    
                    #fileid.write(f"  {out_str}")
                    fileid.write("\n")
    # transpose 
    fileid.write("  !transpose\n")
    for p in range(3):
        fileid.write(f"  K{p}U = transpose(K{p}U); K{p}V = transpose(K{p}V); K{p}W = transpose(K{p}W);\n")

    # end
    fileid.write(f"\nend subroutine get_comp{i + 1}\n\n")


# kernels
fileid.write(f"subroutine get_kernels (NGL,k,{c21f}, &\n")
fileid.write(f"                        U,V,W,Udot,Vdot,Wdot,Kwvnm,&\n")
fileid.write(f"                        {kc21f},dL_dkv) bind(c,name='get_kernels_')\n")
fileid.write("  use iso_c_binding,only: c_int,dp => c_double ,dcp => c_double_complex\n")
fileid.write(f"  implicit none\n\n")
fileid.write(f"  integer(c_int),value,intent(in)         :: NGL\n")
fileid.write(f"  real(dp),value,intent(in)               :: k\n")
fileid.write(f"  real(dp),dimension(NGL),intent(in)      :: {c21f}\n")
fileid.write(f"  complex(dcp),dimension(NGL),intent(in)  :: U,V,W,Udot,Vdot,Wdot\n")
fileid.write(f"  real(dcp),dimension(NGL),intent(out)    :: Kwvnm,{kc21f}\n")
fileid.write(f"  real(dcp),dimension(NGL,2),intent(out)  :: dL_dkv\n")
fileid.write(f"  \n  !local vars\n")
fileid.write(f"  complex(dcp),dimension(NGL)  :: temp\n")
fileid.write(f"  complex(dcp),parameter :: imag_i = cmplx(0.0,1.0,kind=dcp)")
fileid.write(f"\n\n  !init matrices\n")
fileid.write("\n")

# compute kernels d{Lag}/d{Param}
for param in c21vec:
    expr = sp.diff(Lag,param)
    param_out = param 
    if param == k0:
        param_out = 'wvnm'
    f_out = sp.fcode(expr,standard=2008,source_format='free',user_functions=funclist,assign_to=f"temp(:)")
    f_out = replace_fortran_str(f_out)
    fileid.write(f"  {f_out}\n")
    fileid.write(f"  K{param_out}(:) = real(temp(:),kind=dp)\n\n")

# compute group velocity derivative
print(k,c.shape,len(s))
for m in range(2):
    dLag_dkvec = 0
    for i in range(3):
        for j in range(3):
            for p in range(3):
                dLag_dkvec += -I * sp.conjugate(s[i]) * c[i,m,p,j] * (-I * k[j] * s[p] + e3[j] * ds[p])   \
                      - (I * k[j] * sp.conjugate(s[i]) + e3[j] * sp.conjugate(ds[i])) * c[i,j,p,m] * (-I * s[p]) 
    f_out = sp.fcode(dLag_dkvec,standard=2008,source_format='free',user_functions=funclist,assign_to=f"temp(:)")
    f_out = replace_fortran_str(f_out)
    fileid.write(f"  {f_out}\n")
    fileid.write(f"  dL_dkv(:,{m+1}) = real(temp(:),kind=dp)\n")

fileid.write(f"\nend subroutine get_kernels\n\n")



fileid.close()

k^0, Udot psidot = c55
k^0, Wdot psidot = c45
k^0, Vdot psidot = c35
k^1, U psidot = -I*c15*cos(phi) - I*c56*sin(phi)
k^1, Udot psi = I*c15*cos(phi) + I*c56*sin(phi)
k^1, W psidot = -I*c25*sin(phi) - I*c56*cos(phi)
k^1, Wdot psi = I*c14*cos(phi) + I*c46*sin(phi)
k^1, V psidot = -I*c45*sin(phi) - I*c55*cos(phi)
k^1, Vdot psi = I*c13*cos(phi) + I*c36*sin(phi)
k^2, U psi = c11*cos(phi)**2 + 2*c16*sin(phi)*cos(phi) + c66*sin(phi)**2
k^2, W psi = c12*sin(phi)*cos(phi) + c16*cos(phi)**2 + c26*sin(phi)**2 + c66*sin(phi)*cos(phi)
k^2, V psi = c14*sin(phi)*cos(phi) + c15*cos(phi)**2 + c46*sin(phi)**2 + c56*sin(phi)*cos(phi)
k^0, Udot psidot = c45
k^0, Wdot psidot = c44
k^0, Vdot psidot = c34
k^1, U psidot = -I*c14*cos(phi) - I*c46*sin(phi)
k^1, Udot psi = I*c25*sin(phi) + I*c56*cos(phi)
k^1, W psidot = -I*c24*sin(phi) - I*c46*cos(phi)
k^1, Wdot psi = I*c24*sin(phi) + I*c46*cos(phi)
k^1, V psidot = -I*c44*sin(phi) - I*c45*cos(phi)
k^1, Vdot psi = I*c23*sin(phi) + I*c36*cos(phi)
k^2, U psi = c12*